<a href="https://colab.research.google.com/github/chene/ARQOPUS/blob/main/line_Procrustes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Line-based Orthogonal Procrustes Analysis